In [1]:
import cv2
print(cv2.__version__)
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
import imutils
import sys
import numpy
from os import listdir
from os.path import isfile, join
from tqdm import tqdm
import pickle
numpy.set_printoptions(threshold=sys.maxsize)

4.1.1


# Extract marks from images to dictionary and save

In [2]:
path = "/home/maciej/repos/pallet-recognition/data/jpeg_marked/"

def get_marks(filepath):
    marks = []
    img = cv2.imread(filepath)
    validation_sum = 3*255 - 50
    for x,y in np.ndindex(img.shape[:2]):
        if np.sum(img[(x,y)]) > validation_sum:
            marks.append((y,x))
    print(marks)
    return marks

def get_all_marks():
    marks = dict()
    filenames = [f for f in sorted(listdir(path)) if isfile(join(path, f))]
    for i in tqdm(range(len(filenames))):
        filename = filenames[i]
        print(path, filename)
        marks[filename] = get_marks(path + filename)
    return marks

marks = get_all_marks()
np.save(path+"eval/test_marks", marks)


  0%|          | 0/100 [00:00<?, ?it/s]

/home/maciej/repos/pallet-recognition/data/jpeg_marked/ r_1_0.jpg


  1%|          | 1/100 [00:01<02:17,  1.39s/it]

[(495, 264), (310, 266)]
/home/maciej/repos/pallet-recognition/data/jpeg_marked/ r_1_1.jpg


  2%|▏         | 2/100 [00:02<02:18,  1.42s/it]

...

[(424, 311)]
/home/maciej/repos/pallet-recognition/data/jpeg_marked/ r_2_8.jpg


 99%|█████████▉| 99/100 [02:22<00:01,  1.45s/it]

[(418, 315)]
/home/maciej/repos/pallet-recognition/data/jpeg_marked/ r_2_9.jpg


100%|██████████| 100/100 [02:24<00:00,  1.44s/it]

[(397, 318)]


# Evaluate

In [5]:
def dist(a,b):
    return np.linalg.norm(np.array(a)-np.array(b))
    
path = "/home/maciej/repos/pallet-recognition/data/jpeg_marked/eval/"
pred = np.load(path+"pred_centers.npy", allow_pickle=True).item()
test = np.load(path+"test_marks.npy", allow_pickle=True).item()
filenames = test.keys()
FPs = 0
TPs = 0
FNs = 0
acceptance_thr = 15
for filename in filenames:
    marks = test[filename]
    marks_count=len(marks)
    found_marks=set()
    centers = pred[filename]
    for center in centers:
        distances = [dist(center, x) for x in marks]
        if distances:
            #print(distances)
            min_distance_idx = np.argmin(distances)
            min_distance=distances[min_distance_idx]
            if min_distance <= acceptance_thr:
                found_marks.add(min_distance_idx)
            else:
                FPs += 1
    found_marks_count = len(found_marks)
    FNs += marks_count - found_marks_count
    TPs += found_marks_count
    
F1 = 2*TPs/(2*TPs + FPs + FNs)
print(" TPs:",TPs, " FPs:", FPs, " FNs:", FNs, "F1", F1)

 TPs: 75  FPs: 60  FNs: 119 F1 0.45592705167173253
